# Annual Financial Statements - SimFin API

**Luiz Guilherme Gomes Fregona**

12/10/2021

## Table of contents
1. [Introduction](#introduction)
2. [Objetive](#objetive)
3. [Pipeline](#pipeline)
4. [Imports](#imports)
5. [Extract Data from API](#extraction)
    1. [Call the API](#api)
    2. [Save as Dataframe](#save)
7. [Extract, Transform, and Load (ETL)](#etl)
    1. [Set Paths](#paths)
    2. [Extract](#extract)
    3. [Tranform](#transform)
    4. [Load](#load)
    5. [Logging](#logging)
    6. [Running ETL process](#running)

## Introduction <a name="introduction"></a>

The use of data for prediction of stocks has grown in the last 5 years. Both robots and analysts have started to make predictions, and estimates based on financial data, which are regulated and can be found in SEC EDGAR website. In this project, we focused on acquiring financial data from an open-source API called SimFin. The main documents extracted from the API are the income statement, balance sheet, and cash flow of Apple and Google. 

## Objetives <a name="objetive"></a>

The overall goal of this project is to develop an ETL process capable of unifying all three documents into a single .csv file.

## Pipeline <a name="pipeline"></a>

The pipeline decided for this project includes 4 major steps:

1. Extract financial documents from API, and convert each one separetely .csv files
2. Extract .csv files from local repo and store them into a pandas dataframe
3. Tranform data following certain criterias
4. Load the unified dataframe into a final .csv file

## Imports <a name="imports"></a>

Before anything, we are going to set all scientific notation to 5 decimal points

In [108]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

Some libraries will be required before moving forward with this project. The following code check if you already have those libraries.

In [338]:
#Check if the packages are installed in the desktop
try:
    import requests
except ImportError:
    sys.exit("You need the requests package! Run !pip install requests and try again.")
    
try:
    import pandas
except ImportError:
    sys.exit("You need the pandas package! Run !pip install pandas and try again.")

try:
    import os
except ImportError:
    sys.exit("You need the os package! Run !pip install os and try again.")
    
try:
    import numpy
except ImportError:
    sys.exit("You need the os package! Run !pip install numpy and try again.")

try:
    from datetime import datetime
except ImportError:
    sys.exit("You need the os package! Run !pip install datetime and try again.")

try:
    import glob
except ImportError:
    sys.exit("You need the os package! Run !pip install glob and try again.")

In case, you don't have them. Please remove the "#" symbol from the code cells below before running them.

In [5]:
#!pip install requests
#!pip install pandas
#!pip install os
#!pip install numpy
#!pip install datetime
#!pip install glob

## Extract Data from API <a name="extraction"></a>

SimFin is a open-source API which provides financial statements and fundamentals about the main public companies in US. SimFin provides two types of APIs: Python API and WEB API. Here, we are going to use the WEB API. The API key was obtained at https://simfin.com/data/api.

In [7]:
api_key = "f6Dh7PNT3yDf1DRct0XJ2tdeZhY2U8SQ"

As explained before, the goal is to get the income statement, the balance sheet, and the cash flow from apple & google since 2010 until now. 

In [8]:
#Define the stock ticker
tickers = ["AAPL","GOOG"]

#Define the periods and years
period = "fy"
year_start = 2010
year_end = 2020

In [9]:
#Endpoint for all financial statements
request_url = 'https://simfin.com/api/v2/companies/statements'

### Call the API <a name="api"></a>

In order to retrieve matching filings from the API, we are going to send a HTTP POST request to the API with a dictionary formatted payload for each combinating of years, periods, and tickers. The query format follows the recomendations presented at https://simfin.com/api/v2/documentation/#tag/Company/paths/~1companies~1list/get.

In [11]:
#Income Statements

# variable to store the names of the columns
income_columns = []
# variable to store our data
income_data = []

for ticker in tickers:
    
    #loop through all years
    for year in range(year_start, year_end + 1):

                # define the parameters for the query
                parameters = {"statement": "pl", "ticker": ticker, "period": period, "fyear": year, "api-key": api_key}
                # make the request
                request = requests.get(request_url, parameters)

                # convert response to json and take 0th index
                data = request.json()[0]

                # make sure that data was found
                if data['found'] and len(data['data']) > 0:
                    # add the column descriptions once only
                    if len(income_columns) == 0:
                        income_columns = data['columns']
                    # add the data
                    income_data += data['data']

In [12]:
#Balance Sheet

# variable to store the names of the columns
balance_columns = []
# variable to store our data
balance_data = []

for ticker in tickers:
    
    #loop through all years
    for year in range(year_start, year_end + 1):

                # define the parameters for the query
                parameters = {"statement": "bs", "ticker": ticker, "period": period, "fyear": year, "api-key": api_key}
                # make the request
                request = requests.get(request_url, parameters)

                # convert response to json and take 0th index
                data = request.json()[0]

                # make sure that data was found
                if data['found'] and len(data['data']) > 0:
                    # add the column descriptions once only
                    if len(balance_columns) == 0:
                        balance_columns = data['columns']
                    # add the data
                    balance_data += data['data']

In [13]:
#Cash Flow

# variable to store the names of the columns
flow_columns = []
# variable to store our data
flow_data = []

for ticker in tickers:
    
    #loop through all years
    for year in range(year_start, year_end + 1):

                # define the parameters for the query
                parameters = {"statement": "cf", "ticker": ticker, "period": period, "fyear": year, "api-key": api_key}
                # make the request
                request = requests.get(request_url, parameters)

                # convert response to json and take 0th index
                data = request.json()[0]

                # make sure that data was found
                if data['found'] and len(data['data']) > 0:
                    # add the column descriptions once only
                    if len(flow_columns) == 0:
                        flow_columns = data['columns']
                    # add the data
                    flow_data += data['data']

### Save as Dataframe <a name="save"></a>

In order to keep data organized, the data will be stored in .csv files separately.

In [58]:
#Build dataframes
df_income = pd.DataFrame(income_data, columns = income_columns)
df_balance = pd.DataFrame(balance_data, columns = balance_columns)
df_cash_flow = pd.DataFrame(flow_data, columns = flow_columns)

#Check what is the current working directory
os.getcwd()

#Save as .csv files
df_income.to_csv("income.csv", index = False)
df_balance.to_csv("balance.csv", index = False) 
df_cash_flow.to_csv("cash_flow.csv", index = False)

## Extract, Transform, and Load (ETL) <a name="etl"></a>

To keep the whole process of Extract, Tranform, and Load organized, a set of documents and functions will be created first. The documents are a .csv file, where all the information will be stored and read to be uploaded to a relational database, and a .txt logging file to keep track of all steps throughout the ETL process. 

### Set Paths <a name="paths"></a>

In [345]:
logfile    = "logfile.txt"      # all event logs will be stored in this file
targetfile = "final_data.csv"   # file where transformed data is stored

### Extract <a name="extract"></a>

The extract function converts all .csv files into one single dataframe.

In [341]:
#CSV extract function
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

In [342]:
#Files extract function
def extract():
    
    # variable to store the names of the columns
    columns = []
    # variable to store our data
    data = []
    
    #process all csv files
    for csvfile in glob.glob("*.csv"):
            raw_data = extract_from_csv(csvfile)
            extracted_columns = raw_data.columns
            extracted_list = raw_data.transpose().to_numpy().tolist()
            columns.extend(extracted_columns)
            data.extend(extracted_list)

    #Transpose data list
    data = np.array(data).T.tolist()

    #Build final dataframe
    extracted_data = pd.DataFrame(data, columns=columns)
    
    return extracted_data

### Transform <a name="transform"></a>

The transform function does the following tasks:

1. Fill "nan" with 0
2. Get rid of all unnecessary columns
3. Define all datatypes correctly

In [333]:
def transform(data):
        #Fill "nan" values with 0
        data = data.replace("nan", 0)
        
        #Filter duplicated columns
        data = data.loc[:,~data.columns.duplicated()]
        
        #Get rid of useless columns
        useless_col = ["SimFinId","Fiscal Period", "Restated Date", "Publish Date", "Source", "TTM", "Value Check"]
        data = data[data.columns[~data.columns.isin(useless_col)]]
        
        #Convert categorical objects into strings, and date into date
        data['Report Date'] =  pd.to_datetime(data['Report Date'], utc = False)
        data[['Ticker','Fiscal Year']] = data[['Ticker','Fiscal Year']].astype(str)
        
        #The remaining objects were turn into numeric features
        col = data.columns.drop(['Ticker','Fiscal Year','Report Date'])
        data[col] = data[col].apply(pd.to_numeric)
        
        return data

### Load <a name="load"></a>

The load function add the final dataframe to a .csv file ready to be uploaded into a database.

In [335]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  

### Logging <a name="logging"></a>

The logging function help us keep track of all updates performed during the ETL process.

In [336]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

### Running ETL process  <a name="running"></a>

In [339]:
log("ETL Job Started")

In [343]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

,SimFinId,Ticker,Fiscal Period,Fiscal Year,Report Date,Publish Date,Restated Date,Source,TTM,Value Check,...,Common Stock,Additional Paid in Capital,Other Share Capital,Treasury Stock,Retained Earnings,Other Equity,Equity Before Minority Interest,Minority Interest,Total Equity,Total Liabilities & Equity
0,111052,AAPL,FY,2010,2010-09-30,2010-10-27,2012-10-31,https://www.sec.gov/Archives/edgar/data/320193...,False,True,...,nan,nan,nan,nan,37169000000,-46000000,47791000000,nan,47791000000,75183000000
1,111052,AAPL,FY,2011,2011-09-30,2011-10-26,2013-10-30,https://www.sec.gov/Archives/edgar/data/320193...,False,True,...,nan,nan,nan,nan,62841000000,443000000,76615000000,nan,76615000000,116371000000
2,111052,AAPL,FY,2012,2012-09-30,2012-10-31,2014-10-27,https://www.sec.gov/Archives/edgar/data/320193...,False,True,...,nan,nan,nan,nan,101289000000,499000000,118210000000,nan,118210000000,176064000000
3,111052,AAPL,FY,2013,2013-09-30,2013-10-30,2015-10-28,https://www.sec.gov/Archives/edgar/data/320193...,False,True,...,nan,nan,nan,nan,104256000000,-471000000,123549000000,nan,123549000000,207000000000
4,111052,AAPL,FY,2014,2014-09-30,2014-10-27,2016-10-26,https://www.sec.gov/Archives/edgar/data/320193...,False,True,...,nan,nan,nan,nan,87152000000,1082000000,111547000000,nan,111547000000,231839000000
5,111052,AAPL,FY,2015,2015-09-30,2015-10-28,2017-11-03,https://www.sec.gov/Archives/edgar/data/320193...,False,True,...,nan,nan,nan,nan,92284000000,-345000000,119355000000,nan,119355000000,290345000000
6,111052,AAPL,FY,2016,2016-09-30,2016-10-26,2018-11-05,https://www.sec.gov/Archives/edgar/data/320193...,False,True,...,nan,nan,nan,nan,96364000000,634000000,128249000000,nan,128249000000,321686000000
7,111052,AAPL,FY,2017,2017-09-30,2017-11-03,2019-10-31,https://www.sec.gov/Archives/edgar/data/320193...,False,True,...,nan,nan,nan,nan,98330000000,-150000000,134047000000,nan,134047000000,375319000000
8,111052,AAPL,FY,2018,2018-09-30,2018-11-05,2020-10-30,https://www.sec.gov/Archives/edgar/data/320193...,False,True,...,nan,nan,nan,nan,70400000000,-3454000000,107147000000,nan,107147000000,365725000000
9,111052,AAPL,FY,2019,2019-09-30,2019-10-31,2020-10-30,https://www.sec.gov/Archives/edgar/data/320193...,False,True,...,nan,nan,nan,nan,45898000000,-584000000,90488000000,nan,90488000000,338516000000


In [344]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data 

,Ticker,Fiscal Year,Report Date,Revenue,Sales & Services Revenue,Financing Revenue,Other Revenue,Cost of Revenue,Cost of Goods & Services,Cost of Financing Revenue,...,Share Capital & Additional Paid-In Capital,Common Stock,Additional Paid in Capital,Other Share Capital,Treasury Stock,Retained Earnings,Other Equity,Equity Before Minority Interest,Total Equity,Total Liabilities & Equity
0,AAPL,2010,2010-09-30,65225000000,0,0,0,-39541000000,0,0,...,10668000000.00,0,0,0,0,37169000000,-46000000,47791000000,47791000000,75183000000
1,AAPL,2011,2011-09-30,108249000000,0,0,0,-64431000000,0,0,...,13331000000.00,0,0,0,0,62841000000,443000000,76615000000,76615000000,116371000000
2,AAPL,2012,2012-09-30,156508000000,0,0,0,-87846000000,0,0,...,16422000000.00,0,0,0,0,101289000000,499000000,118210000000,118210000000,176064000000
3,AAPL,2013,2013-09-30,170910000000,0,0,0,-106606000000,0,0,...,19764000000.00,0,0,0,0,104256000000,-471000000,123549000000,123549000000,207000000000
4,AAPL,2014,2014-09-30,182795000000,0,0,0,-112258000000,0,0,...,23313000000.00,0,0,0,0,87152000000,1082000000,111547000000,111547000000,231839000000
5,AAPL,2015,2015-09-30,233715000000,0,0,0,-140089000000,0,0,...,27416000000.00,0,0,0,0,92284000000,-345000000,119355000000,119355000000,290345000000
6,AAPL,2016,2016-09-30,215639000000,0,0,0,-131376000000,0,0,...,31251000000.00,0,0,0,0,96364000000,634000000,128249000000,128249000000,321686000000
7,AAPL,2017,2017-09-30,229234000000,0,0,0,-141048000000,0,0,...,35867000000.00,0,0,0,0,98330000000,-150000000,134047000000,134047000000,375319000000
8,AAPL,2018,2018-09-30,265595000000,0,0,0,-163756000000,0,0,...,40201000000.00,0,0,0,0,70400000000,-3454000000,107147000000,107147000000,365725000000
9,AAPL,2019,2019-09-30,260174000000,0,0,0,-161782000000,0,0,...,45174000000.00,0,0,0,0,45898000000,-584000000,90488000000,90488000000,338516000000


In [346]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

In [347]:
log("ETL Job Ended")